In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy  as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline
from catboost import CatBoostClassifier, FeaturesData
import datetime as DT
import io
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


Importando arquivos

In [3]:
tabelao = pd.read_csv('tabelao_idade.csv')

In [4]:
tabelao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96406 entries, 0 to 96405
Data columns (total 21 columns):
CODIGO_CLIENTE             96406 non-null int64
DATA_NASCIMENTO            96403 non-null object
IDADE                      96406 non-null int64
SEXO                       96405 non-null object
ESTADO_CIVIL               96405 non-null object
CIDADE                     96406 non-null object
UF                         96406 non-null object
BAIRRO                     96406 non-null object
CEP                        96406 non-null int64
QTD_DEPENDENTES            96405 non-null float64
DATA_CADASTRO              96406 non-null object
DATA_ADMISSAO              52050 non-null object
CATEGORIAL_PROFISSAO       94690 non-null object
TIPO_RESIDENCIA            95454 non-null object
RENDA_TITULAR              96400 non-null float64
RENDA_CJ                   4393 non-null float64
OUTRAS_RENDAS              11070 non-null float64
QTD_CARTOES_ADICIONAIS     96406 non-null int64
FLAG_CONTA

In [5]:
tabelao = tabelao.drop(columns=['CODIGO_CLIENTE', 'DATA_NASCIMENTO','CIDADE','UF', 'BAIRRO','CEP', 'DATA_CADASTRO','DATA_ADMISSAO',
             'RENDA_CJ','OUTRAS_RENDAS','FLAG_CONTA_BANCO',
                                'QTD_CARTOES_ADICIONAIS','VALOR_HISTORICO_COMPRAS'], axis=1)

In [6]:
tabelao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96406 entries, 0 to 96405
Data columns (total 8 columns):
IDADE                   96406 non-null int64
SEXO                    96405 non-null object
ESTADO_CIVIL            96405 non-null object
QTD_DEPENDENTES         96405 non-null float64
CATEGORIAL_PROFISSAO    94690 non-null object
TIPO_RESIDENCIA         95454 non-null object
RENDA_TITULAR           96400 non-null float64
CLASS                   96406 non-null int64
dtypes: float64(2), int64(2), object(4)
memory usage: 5.9+ MB


In [7]:
tabelao.head()

,IDADE,SEXO,ESTADO_CIVIL,QTD_DEPENDENTES,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,CLASS
0,63,M,SO,0.0,APOSENTADO,PR,2300.0,1
1,51,M,CA,2.0,ASSALARIADO,OU,1500.0,1
2,30,M,SO,0.0,AUT+NOMO,PR,1000.0,1
3,65,F,SO,0.0,APOSENTADO,OU,1050.0,1
4,62,F,SO,0.0,AUT+NOMO,PR,1200.0,1


In [8]:
tabelao.CLASS.value_counts()

1    72127
0    24279
Name: CLASS, dtype: int64

In [9]:
tabelao = pd.get_dummies(tabelao, columns=['SEXO', 'ESTADO_CIVIL', 'CATEGORIAL_PROFISSAO',
                                           'TIPO_RESIDENCIA'], drop_first=True)

In [10]:
tabelao = tabelao.dropna()

In [11]:
# Verificar qual a coluna 'CLASS'
X = tabelao.iloc[:, :].values
X = np.delete(X, obj= 3, axis= 1)
y = tabelao.iloc[:, 3].values

In [12]:
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[    0     1]
 [24277 72123]]


In [13]:
tabelao.head()

,IDADE,QTD_DEPENDENTES,RENDA_TITULAR,CLASS,SEXO_M,ESTADO_CIVIL_CO,ESTADO_CIVIL_SE,ESTADO_CIVIL_SO,ESTADO_CIVIL_VI,CATEGORIAL_PROFISSAO_APOSENTADO,CATEGORIAL_PROFISSAO_ASSALARIADO,CATEGORIAL_PROFISSAO_AUT+NOMO,CATEGORIAL_PROFISSAO_LIBERAL,TIPO_RESIDENCIA_FA,TIPO_RESIDENCIA_FI,TIPO_RESIDENCIA_OU,TIPO_RESIDENCIA_PR,TIPO_RESIDENCIA_SM
0,63,0.0,2300.0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0
1,51,2.0,1500.0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0
2,30,0.0,1000.0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,0
3,65,0.0,1050.0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0
4,62,0.0,1200.0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0


In [17]:
tabelao.to_csv('tabela_final.csv', index=False)

In [14]:
#skf = StratifiedKFold(n_splits=5,)

In [15]:
# for train_index, test_index in skf.split(X, y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     sm = SMOTE()
#     X_res, y_res = sm.fit_resample(X_train, y_train)
    
#     #LR
#     lr= LogisticRegression()
#     lr.fit(X_res, y_res)
    
#     #SVM
#     clf = svm.SVC()
#     clf.fit(X_res, y_res)
    
#     y_pred = clf.predict(X_test)
#     cm = confusion_matrix(y_test, y_pred)
#     print(cm)
#     print(classification_report(y_test, y_pred))